# 🌴 Asistente Turístico de Tenerife - RAG Demo

## 1. Configuración del Entorno
Cargamos las librerías necesarias y verificamos la API Key.

In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    print(f"✅ Key cargada: {api_key[:5]}...")
else:
    print("❌ Error de Key")

✅ Key cargada: sk-pr...


## 2. Validación de Conectividad con OpenAI
Probamos que la llave funcione y el modelo responda.

In [2]:
from langchain_openai import ChatOpenAI

# Inicializamos el modelo
llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

# Prueba 'Holística'
try:
    response = llm.invoke("Di 'Hola Tenerife' si me escuchas.")
    print(f"🤖 Respuesta Modelo: {response.content}")
except Exception as e:
    print(f"❌ Error de Conexión: {e}")

🤖 Respuesta Modelo: Hola Tenerife. ¿En qué puedo ayudarte hoy?


## 3. Fase 2: Ingesta y Vector Store 📚
Subimos el PDF `TENERIFE.pdf` a OpenAI para crear un Vector Store gestionado.

In [3]:
import os
import openai
from openai import OpenAI
from pathlib import Path

# DEBUG: Verificar entorno
print(f"📂 CWD: {os.getcwd()}")
print(f"📦 OpenAI Version: {openai.__version__}")

client = OpenAI()

# Path corregido y validado
pdf_path = Path("../data/raw/TENERIFE.pdf")

if not pdf_path.exists():
    print(f"❌ No encontrado: {pdf_path}")
else:
    print(f"✅ Archivo encontrado: {pdf_path}")

    # Crear Vector Store
    try:
        if not hasattr(client, 'beta'):
            print("❌ La versión de OpenAI instalada no tiene acceso a 'beta'. Actualiza la librería.")
        else:
            vector_store = client.beta.vector_stores.create(name="Tenerife Guide Store")
            print(f"📦 Vector Store creado: {vector_store.id}")
            
            # Subir archivos
            with open(pdf_path, "rb") as f:
                file_batch = client.beta.vector_stores.file_batches.upload_and_poll(
                    vector_store_id=vector_store.id, files=[f]
                )
            print(f"📄 Estado de carga: {file_batch.status}")
            print(f"🔢 Archivos procesados: {file_batch.file_counts}")
    except Exception as e:
        print(f"❌ Error Critical: {e}")

📂 CWD: f:\development\Development\Master IA\LLM-large-language-models-entrega-tarea-final\notebooks
📦 OpenAI Version: 1.59.3
✅ Archivo encontrado: ..\data\raw\TENERIFE.pdf
📦 Vector Store creado: vs_69661d46318481919de2cb885703f8d0
📄 Estado de carga: completed
🔢 Archivos procesados: FileCounts(cancelled=0, completed=1, failed=0, in_progress=0, total=1)
